# Data Science in Psychology & Neuroscience (DSPN): 

## Lecture 13. Data Wrangling (part 3)

### Date: October 4, 2022

### To-Dos From Last Class:

* Download "imitation inhibition" task data from <a href="https://github.com/hogeveen-lab/DSPN_Fall2022_Git/tree/master/misc_exercises/imitation_inhibition_paradigm">Github</a>
* Download <a href="https://github.com/hogeveen-lab/DSPN_Fall2022_Git/tree/master/assignment_starters/assign3_starter">Assignment #3 starter kit</a>

### Today:

* Wrangle some real data

### Homework
* Submit Assignment #3: <a href="https://www.dropbox.com/request/5tyrxS7O80R8XD2qLbx5">Data Wrangling</a> (before 10/13, 23:00 MDT)
* Open R Studio from anaconda (optional: <a href="https://richpauloo.github.io/2018-05-16-Installing-the-R-kernel-in-Jupyter-Lab/">set up an R Kernel to stay in Jupyter</a>; warning: can come with some package installation headaches!)

# Importing packages

# Setting up the filepaths

# Loading in a test subject to make sense of things

# Iterating through to get all first level data, concatenating into a single data frame

# Loading in the questionnaire data and merging it with behavioral data

# Writing the observation-level data to a big csv

## Quick tangent: Automatic Imitation Task

<img src="img/ait_task.png">

* __MAIN OUTCOME: HOW DO THE DIFFERENT CONGRUENT VS. INCONGRUENT CONDIS SLOW YOUR RESPONSE TIME?__

# Computing SECOND-level summary measures (mean RT by pid)

# Computing SECOND-level summary measures (mean RT by pid by condition)

# Writing the subject-level RT data to a CSV